In [33]:
from tqdm.auto import tqdm
import time
import subprocess
import random

In [34]:
wa = False

class Test():
    NODES = 10000
    CONNECTIONS = 200_000
    
    def __init__(self):
        self.n = self.NODES
        self.generate_test()
        
    def generate_test(self): 
        with open("test.in", 'w') as test:
            test.write(f"{self.CONNECTIONS}\n")
            
            con = {}
            self.G = [set() for _ in range(self.n + 1)]
            
            for _ in range(self.CONNECTIONS):
                a = random.randint(0, self.n - 1)
                b = random.randint(0, self.n - 1)
                
                no_conn = con.get((a, b), 0)
                self.G[a].add((b, no_conn))
                
                test.write(f"{a} {no_conn} {b}\n")
                con[(a, b)] = no_conn + 1
        
    def check_solutions(self, a_out, b_out):
        global wa
        # a: brut, b: better_solution
        with open(a_out, 'r') as a_out, open(b_out, 'r') as b_out:
            first_line_a = a_out.readline().strip()
            first_line_b = b_out.readline().strip()
            
            if first_line_a == "BRAK" == first_line_b:
                return 
            
            if first_line_a == "BRAK" or first_line_b == "BRAK" and first_line_a != first_line_b:
                print("WA")
                wa = True
                return
            
            # better check brut 
            self.test_n = int(first_line_a)
            self.test_data = []
            for _ in range(self.test_n):
                a, b, c = map(int, a_out.readline().strip().split())
                self.test_data.append((a, b, c))
                
            self.better_check()
            
            # better check giga_chad
            self.test_n = int(first_line_b)
            self.test_data = []
            for _ in range(self.test_n):
                a, b, c = map(int, b_out.readline().strip().split())
                self.test_data.append((a, b, c))
                
            self.better_check()
    
    def better_check(self):
        global wa
        node: int = 0
        used_edges = {}
        
        for a, idx, b in self.test_data:
            if a != node:
               print("WA")  
               wa = True
               return
               
            if (a, idx, b) in used_edges:
               print("WA")  
               wa = True
               return
               
            used_edges[(a, idx, b)] = 1
            
            if not (b, idx) in self.G[a]:
                print("WA")
                wa = True
                return
                
            node = b 
            
        if node != 0:
            print("WA")
            wa = True
            return

In [ ]:
# convention a: brut, b: better_solutoin
class Referee():
    def __init__(self, program_a: str, program_b: str, max_time):
        # program a = sth.cpp (give full name with .cpp)
        
        self.program_a = program_a
        self.program_b = program_b
        self.max_time = max_time
        
        # compile programs 
        self.exe_a = self.compile(program_a)
        self.exe_b = self.compile(program_b)
        
    def compile(self, program: str) -> str:
        exe_name = program.split('.')[0]
        compile_command = ['g++', program, '-o', exe_name]
        subprocess.run(compile_command, check=True)
        return exe_name
    
    def do_tests(self): 
        self.times = []
        for _ in tqdm(range(200)): 
            # create test 
            self.test = Test()
            
            # test program_a
            start_time_a = time.time()
            with open("test.in", "r") as test_input, open(f"{self.exe_a}.out", "w") as output_a:
                try:
                    subprocess.run(f"./{self.exe_a}", stdin=test_input, stdout=output_a, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_a} exceeded time limit")
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_a} ended with an error")
                    exit(2)
            end_time_a = time.time()
            
            # test program_b
            start_time_b = time.time()
            with open("test.in", "r") as test_input, open(f"{self.exe_b}.out", "w") as output_b:
                try:
                    subprocess.run(f"./{self.exe_a}", stdin=test_input, stdout=output_b, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_b} exceeded time limit") 
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_b} ended with an error")
                    exit(2)
            end_time_b = time.time()
            
            
            # check solution
            global wa
            wa = False
            self.test.check_solutions(f"{self.exe_a}.out", f"{self.exe_b}.out")
            if wa == True:
               break 
            
            # solutions are ok
            self.times.append([end_time_a - start_time_a, end_time_b - start_time_b])
        
        else:
            print("everything ok 🚀") 

In [36]:
referee = Referee('brut_solution.cpp', 'gigachad_solution.cpp', 2)
referee.do_tests()

  0%|          | 0/20 [00:00<?, ?it/s]

everything ok 🚀


In [47]:
import pandas as pd
df = pd.DataFrame(referee.times)
df.columns = ['brut', 'chad']
df = df.sort_values(by='chad', ascending=False)
df

,brut,chad
12,0.064821,0.064690
15,0.064685,0.064687
5,0.064831,0.064653
1,0.064538,0.064622
8,0.064589,0.064610
18,0.064375,0.064562
9,0.064565,0.064559
4,0.064540,0.064554
6,0.064852,0.064545
10,0.064568,0.064523
